In [1]:
import anndata
import seaborn as sns
import matplotlib.pyplot as plt

import sopa
import sopa.spatial

heatmap_kwargs = {"vmax": 40, "cmap": sns.cm.rocket_r, "cbar_kws": {'label': 'Mean hop distance'}}

## 1. Prepare your data

You'll need the `AnnData` output of Sopa. If using the `SpatialData` object itself, simply extract the table.

Make sure you have at least a cell-type annotation (i.e. a column in `adata.obs` corresponding to cell-types), and eventually a niche annotation (with algorithms such as [STAGATE](https://github.com/zhanglabtools/STAGATE)).

#### (Optional) Download the tutorial data

The `.h5ad` file used in this tutorial is publicly available on Zenodo [here](https://doi.org/10.5281/zenodo.10512440).

In [2]:
adata = anndata.read_h5ad("adata_liver_merscope.h5ad")

Then, compute the Delaunay graph on your data. Especially, use the `radius` argument to drop long edges. In this examples, edges longer than 50 microns are removed.
> The later function comes from [Squidpy](https://squidpy.readthedocs.io/en/latest/api/squidpy.gr.spatial_neighbors.html#squidpy.gr.spatial_neighbors).

In [3]:
sopa.spatial.spatial_neighbors(adata, radius=[0, 50])

## 2. Distances between cell categories

You can compute the mean hop-distance between all pairs of cell-types:
> Below, `'cell_type'` is the name of the column of `adata.obs` containing the cell-type annotation

In [4]:
cell_type_to_cell_type = sopa.spatial.mean_distance(adata, "cell_type", "cell_type")

In [5]:
plt.figure(figsize=(7, 6))
sns.heatmap(cell_type_to_cell_type, **heatmap_kwargs)

Similary, you can compute the mean hop-distance between all pairs of cell-types and niches:

In [6]:
cell_type_to_niche = sopa.spatial.mean_distance(adata, "cell_type", "niches")

In [7]:
plt.figure(figsize=(3, 6))
sns.heatmap(cell_type_to_niche, **heatmap_kwargs)

Same between niches and niches:

In [8]:
niche_to_niche = sopa.spatial.mean_distance(adata, "niches")

In [9]:
plt.figure(figsize=(3, 3))
sns.heatmap(niche_to_niche, **heatmap_kwargs)

## 3. Transform niches into shapes
If desired, niches can be transformed into [Shapely](https://shapely.readthedocs.io/en/stable/index.html) geometries. Each occurence of a specific niche will correspond to one Polygon. This makes efficient further operations on niches, such as the one in the next section.

In [10]:
gdf = sopa.spatial.geometrize_niches(adata, "niches")
gdf

Now, each occurence (or connected component) of each niche category is a Polygon. On this example, the Necrosis niche has 3 components, as shown below.

In [11]:
legend_kwds = {"bbox_to_anchor": (1.04, 0.5), "loc": "center left", "borderaxespad": 0, "frameon": False, "title": "Niches"}

gdf.plot(column="niches", legend=True, legend_kwds=legend_kwds)

## 4. Niches geometries
For each niche, we can compute geometric properties. Here, we computed some simple properties of each niche: their mean length (or perimeter), their mean area, and their mean roundness (score between 0 and 1, where high values means "circle"-like shape).

> NB: Since one niche can be divided into multiple connected components (or multiple occurences), we indeed need to average the above geometric properties over all connected components of one niche category

In [12]:
df_niches_geometries = sopa.spatial.niches_geometry_stats(adata, "niches")
df_niches_geometries

In [13]:
fig, axes = plt.subplots(1, 4, figsize=(15, 6))

for i, name in enumerate(["n_components", "length", "area", "roundness"]):
    vmax = df_niches_geometries[name].sort_values()[-2:].mean()
    sns.heatmap(df_niches_geometries[[name]], cmap="viridis", annot=True, fmt=".2f", vmax=vmax, ax=axes[i])

plt.subplots_adjust(wspace=1.5)

## 5. Cell-type / Niche network

The distances between cell-types and/or niches can be summerized into one network, and plot with the [Netgraph](https://netgraph.readthedocs.io/en/latest/index.html) library. It provides a quick overview of the interactions happening in the micro-environment of one slide.

To continue, you'll need to install Louvain and Netgraph:

```sh
!pip install python-louvain
!pip install netgraph
```

In [14]:
import networkx as nx
from community import community_louvain
from netgraph import Graph

In [15]:
weights, node_color, node_size, node_shape = sopa.spatial.prepare_network(adata, "cell_type", "niches")

In [16]:
g = nx.from_pandas_adjacency(weights)
node_to_community = community_louvain.best_partition(g, resolution=1.35)

In [17]:
Graph(g,
      node_size=node_size,
      node_color=node_color,
      node_shape=node_shape,
      node_edge_width=0,
      node_layout='community',
      node_layout_kwargs=dict(node_to_community=node_to_community),
      node_labels=True,
      node_label_fontdict=dict(size=6, weight="bold"),
      edge_alpha=1,
      edge_width=0.5,
      edge_layout_kwargs=dict(k=2000),
      edge_layout='bundled',
)